## PRACTICA OBTENCIÓN DE DATOS

### Web scraper de la página: http://gestiona.madrid.org/azul_internet/html/web/DatosEstacion24Accion.icm?ESTADO_MENU=2_1

#### Libraries

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

---

#### Vamos a hacer una primera exploración rápida de la página, para entender su estructura y lo que necesitamos

In [5]:
#Primero importamos el html de la página web
website = requests.get("http://gestiona.madrid.org/azul_internet/html/web/DatosEstacion24Accion.icm?ESTADO_MENU=2_1")

#Ahora lo guardamos con un objeto "soup"
website_soup = bs(website.content)

#Finalmente imprimimos para ver que tenemos
print(website_soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
 <head>
  <title>
   madrid.org - Comunidad de Madrid
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="es" http-equiv="Content-Language"/>
  <meta content="Web oficial del gobierno autonómico con información sobre economía, educación, servicios sociales,..." name="Description"/>
  <meta content="Comunidad de Madrid" name="Keywords"/>
  <meta content="Comunidad de Madrid" name="Author"/>
  <meta content="oficina.atencion.ciudadano@madrid.org" name="Owner"/>
  <!--link rel="stylesheet" type="text/css" href="../../style/intranet.css"-->
  <link href="../../style/sistemas.css" rel="stylesheet" type="text/css"/>
  <link href="../../style/estilos.css" rel="stylesheet" type="text/css"/>
  <link href="../../style/solapas.css" rel="stylesheet" type="text/css"/>
  <script language="JavaScript" src="../../js/solapas.js" type="text/javascri

#### Usando la herramienta inspeccionar, vemos que las tablas que nos interesa scrapear están dentro de etiquetas "table"

In [7]:
#Vamos a probar con un find all
tables = website_soup.find_all("table")
print(tables)

#Esto ha creado una lista de todas las tablas

[<table border="0" cellpadding="0" cellspacing="0" style="width:980px">
<tr>
<td style="padding-bottom: 10px;" width="89">
<!--      <img style="cursor: pointer;" -->
<!--       		src="../../images/portal/PubliPositivo_15mm_ancho.jpg" alt="Comunidad de Madrid - madrid.org" title="Comunidad de Madrid - madrid.org" width="89" height="64"/> -->
<a href="http://www.comunidad.madrid" target="_blank">
<img alt="Comunidad de Madrid" height="64" src="/webutils/logoCM-izq-89x64.png" style="cursor: pointer;" title="Comunidad de Madrid" width="89"/>
</a>
</td>
<td class="nihil" width="15">
       
    </td>
<td>
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<!--         <tr> -->
<!--           <td colspan="2" class="txt08gr3"> -->
<!--           </td> -->
<!--         </tr> -->
<tr>
<td class="titGeo11azu">
            Red de Calidad del Aire de la Comunidad de Madrid
          </td>
<td id="tdalerta">
</td>
</tr>
<tr>
<td class="puntos" colspan="2">
</td>
</tr>
<tr>
<td colspan

#### Vemos que hay 6 tablas en la página, y comprobamos que las que nos interesan son la nº 3 y la nº4 ( [2] y [3])

In [16]:
#Tabla 1_ es la número tres de la lista y por tanto [2]
table_one = website_soup.select("table")[2]
print(table_one.prettify())

<table align="center" border="0" cellpadding="4" cellspacing="1" width="94%">
 <thead>
  <tr>
   <td align="center" class="txt08gr3" colspan="8" id="fondoVainilla">
    Ultimas 24 Horas
   </td>
  </tr>
  <tr align="center" valign="middle">
   <td>
   </td>
   <td align="center" class="txt08bla" colspan="7" id="fondoGrisMedio">
    <strong>
     Contaminantes
    </strong>
   </td>
  </tr>
  <tr>
   <td>
   </td>
   <td align="center" class="txt07azu" id="fondoGris">
    <a href="javascript:abrirVentana('DatosEstacionGraficoAccion.icm?idAnalizador=7&amp;limSup=24&amp;limInf=0&amp;fechaIni=2021/12/25&amp;fechaFin=2021/12/26&amp;estacion=Getafe&amp;analizador=Temperatura interior cabina', 'wcTIN', 720, 480);" title="Temperatura interior cabina. Pulse con el ratón para ver la gráfica de evolución.">
     <strong>
      TIN
      <br/>
     </strong>
     <small>
      (ºC)
     </small>
    </a>
   </td>
   <td align="center" class="txt07azu" id="fondoGris">
    <a href="javascript:abrirV

### Ahora vamos a definir las funciones de nuestro código

#### Functions

In [17]:
def get_html(url, station_num):
    """
    Función para extraer un objeto soup con el HTML de la url introducida.
    :url: string con la url de la web de madrid.org.
    :return: devuelve el objeto soup.
    """
    # Primero importamos el html de la página web
    website = requests_url(url, station_num)
    
    # Ahora lo guardamos con un objeto "soup"
    website_soup = bs(website.content)
    return website_soup

def get_tables(soup):
    """
    Función master para extraer las tablas del HTML.
    :soup: string con el objeto soup HTML.
    :return: devuelve una tupla con los dos dataframes extraídos.
    """
    # Vamos a probar con un find all
    tables = soup.find_all("table")
    df_tables = ()
    # Seleccionamos las tablas 2 y 3 de la página a escrapear
    for table in tables[2:4]:
        df = table_to_pandas(table)
        df_tables += (df,)
    return df_tables
        
def table_to_pandas(table_soup):
    """
    Función auxiliar para parsear una tabla HTML y convertirla a Dataframe.
    :table_soup: string con el html de la tabla.
    :return: dataframe con la tabla.
    """
    # Vamos a coger los nombres de las columnas de la tabla
    head = table_soup.find("thead").find_all("strong")[1:]
    columns = [column.text for column in head]
    columns.insert(0, 'HOURS')
    
    # Vamos a coger las filas de la tabla
    rows = []
    body = table_soup.find("tbody").find_all("tr")
    for row in body:
        values = row.find_all("td")
        values = [value.text.strip() for value in values]
        rows.append(values)
    
    # Construimos el dataframe
    df = pd.DataFrame(rows, columns = columns)
    return df

def requests_url(url, station_num):
    """
    Función para extraer el correcto html en función de la estación.
    :url: string con la url de la web de madrid.org.
    :station_num: integer con el número de estación a consultar.
    :return: devuelve la html de respuesta con las condiciones establecidas.
    """
    url = "http://gestiona.madrid.org/azul_internet/html/web/DatosEstacion24Accion.icm?ESTADO_MENU=2_1"

    # Aqui establecemos la opción de introducir el número de estación concreto a consultar.
    payload='estaciones=' + str(station_num) + '&accept=Accept'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
  
    # Guardamos el request en la variable response
    response = requests.request("POST", url, headers=headers, data=payload)

    return response


#### CODE

In [19]:
# CODE
# Url con los datos a scrapear
url = "http://gestiona.madrid.org/azul_internet/html/web/DatosEstacion24Accion.icm?ESTADO_MENU=2_1"
station_num = 18

# Extracción de tablas del HTML
html = get_html(url, station_num)
pollution, weather = get_tables(html)

# Mostrar datos extraídos
display(pollution.head())
display(weather.head())

,HOURS,TIN,NO,NO2,PM10,NOX,O3
0,23:00,18.4,3,29,10,34,25
1,24:00,18.2,2,21,11,24,30
2,01:00,18.3,1,19,10,21,30
3,02:00,18.3,2,25,12,28,21
4,03:00,18.2,1,27,16,29,15


,HOURS,VV,DV,TMP,HR,PRE,RS,LL
0,23:00,0.7,338,9.8,95,941,1,0.20
1,24:00,0.3,84,9.8,95,941,1,0.00
2,01:00,0.7,201,9.7,95,941,1,0.00
3,02:00,0.3,231,9.7,95,942,1,0.00
4,03:00,0.9,183,9.5,96,943,1,0.00


---